In [ ]:
from sklearn.datasets import make_regression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
# costante seme per gli algoritmi randomici
SEED = 42
NUMBER_OF_FEATURES = 50
# costruisce la metrica a partire da funzioni per scalcolare la metrica elencate nella documentazione
# nella pagina di ingresso di sklearn.metrics
myscorer = make_scorer(mean_squared_error)

In [ ]:
"""
# classificazione
X_created, y_created = make_classification(n_samples=100, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=SEED, shuffle=False)
X_created
"""

'\n# classificazione\nX_created, y_created = make_classification(n_samples=100, n_features=4,\n                           n_informative=2, n_redundant=0,\n                           random_state=SEED, shuffle=False)\nX_created\n'

In [ ]:
X_created, y_created = make_regression(n_samples=1000, n_features=50,random_state = SEED)
X_created

array([[ 0.82004809,  0.46818219,  1.01440426, ...,  0.04068039,
         0.2128115 ,  1.17283628],
       [-0.39955306,  1.22978153,  0.52828833, ...,  0.09898649,
         0.19278408,  0.87713462],
       [-1.21117193,  0.68631846, -0.31305803, ..., -1.86653995,
         0.35561335,  0.10435594],
       ...,
       [-0.94835411,  1.66303138, -0.26890768, ..., -0.43915533,
         1.72388488, -0.62214095],
       [ 1.47431294, -0.49244138,  0.36926553, ..., -1.33949216,
         0.84887383,  1.49805028],
       [ 0.24387986,  0.91282816,  1.24558824, ..., -0.63019464,
        -0.14288297, -1.591054  ]])

In [ ]:
X, X_new, y, y_new = train_test_split(X_created, y_created, test_size=0.3, random_state=SEED)
# per la classificazione aggiungere stratify = y
# X, X_new, y, y_new = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
pipeSVR = make_pipeline(# eventualemente passo di inputazione, l'imputazione avverrà solo sulle Xtrain e non su Xtrain, ytrain. Come anche per lo scaling
                        # Se facessimo classificazione e al posto di PCA(), avessimo LDA() questa, essendo supervisionata, userebbe le coppie Xtrain, ytrain
                     StandardScaler(),
                     PCA(n_components=NUMBER_OF_FEATURES),
                     LinearSVR(max_iter=10000))
pipeRR = make_pipeline(# eventualemente passo di inputazione l'imputazione avverrà solo sulle Xtrain e non su Xtrain, ytrain. Come anche per lo scaling
                       # Se facessimo classificazione e al posto di PCA(), avessimo LDA() questa, essendo supervisionata, userebbe le coppie Xtrain, ytrain
                     StandardScaler(),
                     PCA(n_components=NUMBER_OF_FEATURES),
                     Ridge())

In [ ]:
print(pipeSVR.named_steps)
print(pipeRR.named_steps)

{'standardscaler': StandardScaler(), 'pca': PCA(n_components=50), 'linearsvr': LinearSVR(max_iter=10000)}
{'standardscaler': StandardScaler(), 'pca': PCA(n_components=50), 'ridge': Ridge()}


In [ ]:
# parametro di regolarizzazione
# shuffle = True, Whether to shuffle each class’s samples before splitting into batches. Note that the samples within each split will not be shuffled.
# se fosse classificazione al posto di KFlod metti StratifiedKFold
outer_cv = KFold(n_splits=10, shuffle=True, random_state= SEED)
param_gridSVR = {
    'linearsvr__C': [0.1, 1, 10],
    #'epsilon': [0.01, 0.1, 1]
}
param_gridRR = {
    'ridge__alpha': [0.1, 1, 10, 100],
    #'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sag']
}


In [ ]:
outer_resultsRR = []
outer_best_estimatorsRR = []
outer_best_paramsRR = []

outer_resultsSVR = []
outer_best_estimatorsSVR = []
outer_best_paramsSVR = []
# se stessimo usando StratifiedKFold (per la classificazione) outer_cv.split(X, y)
for train_idx, test_idx in outer_cv.split(X):
    # Split data into training and test sets for the current fold
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Define the inner cross-validation loop and GridSearchCV
    # se fosse classificazione al posto di KFlod metti StratifiedKFold
    inner_cv = KFold(n_splits=5, shuffle=True, random_state=SEED)
    grid_searchSVR = GridSearchCV(estimator=pipeSVR,
                               # estimator=LinearSVR(max_iter=10000),
                               param_grid=param_gridSVR,
                               cv=inner_cv,
                               scoring=myscorer,
                               # scoring='neg_mean_absolute_error',
                               refit = True)

    grid_searchRR = GridSearchCV( estimator=pipeRR,
                               # estimator=Ridge(),
                               param_grid=param_gridRR,
                               cv=inner_cv,
                               scoring=myscorer,
                               # scoring='neg_mean_absolute_error',
                               refit = True)

    # Fit the GridSearchCV
    grid_searchSVR.fit(X_train, y_train)
    grid_searchRR.fit(X_train, y_train)

    # Retrieve and store the best model from GridSearchCV
    best_modelSVR = grid_searchSVR.best_estimator_
    best_modelRR = grid_searchRR.best_estimator_
    outer_best_estimatorsSVR.append(best_modelSVR)
    outer_best_estimatorsRR.append(best_modelRR)
    # # Retrieve and store the best param from GridSearchCV
    best_paramSVR = grid_searchSVR.best_params_
    best_paramRR = grid_searchRR.best_params_
    outer_best_paramsSVR.append(best_paramSVR)
    outer_best_paramsRR.append(best_paramRR)
    # Print the best parameters and score
    print(grid_searchSVR.best_score_)
    print(grid_searchRR.best_score_)

    # Evaluate the best model on the outer validation set
    y_predSVR = best_modelSVR.predict(X_test)
    y_predRR = best_modelRR.predict(X_test)

   # maeSVR = mean_absolute_error(y_test, y_predSVR)
   # maeRR = mean_absolute_error(y_test, y_predRR)
    maeSVR = myscorer(best_modelSVR, X_test, y_test)
    maeRR = myscorer(best_modelRR, X_test, y_test)

    outer_resultsSVR.append(maeSVR)
    outer_resultsRR.append(maeRR)


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: Futu

16196.997444263476
850.1827294298919


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: Futu

16018.01088275878
853.5173154200627


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: Futu

...

15744.11030306279
846.408437346644


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: Futu

16257.258213535937
827.8303242967546


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: Futu

16173.141590192396
807.8710994206673


In [ ]:
print(outer_resultsSVR)
print(outer_resultsRR)

[14269.797838290724, 14850.65348385879, 14116.28697063412, 13599.942535265009, 12762.028123971386, 12255.687974566363, 14099.433772372504, 16904.439175868265, 11555.77542194095, 11319.005371436999]
[511.4476948363408, 625.7755108883258, 592.9251818148964, 555.8652116206598, 476.68025760777255, 520.1287813714619, 594.1791954978708, 764.0661010268385, 480.31036659332085, 460.54300617169247]


In [ ]:
print("Errore di training-validazione per il modello1: %.2f +/- %.2f" % (np.mean(outer_resultsSVR), np.std(outer_resultsSVR)))
print("Errore di training-validazione per il modello2: %.2f +/- %.2f" % (np.mean(outer_resultsRR), np.std(outer_resultsRR)))

Errore di training-validazione per il modello1: 13573.31 +/- 1591.61
Errore di training-validazione per il modello2: 558.19 +/- 86.80


In [ ]:
# tra i migliori modelli per ogni fold, scelgi il modello migliore
final_modelSVR = outer_best_estimatorsSVR[np.argmin(outer_resultsSVR)]
final_paramSVR = outer_best_paramsSVR[np.argmin(outer_resultsSVR)]
final_modelRR = outer_best_estimatorsRR[np.argmin(outer_resultsRR)]
final_paramRR = outer_best_paramsRR[np.argmin(outer_resultsRR)]
print("I migliori parametri in assoluto per modello1 sono: ")
print(final_paramSVR)
print("I migliori parametri in assoluto per modello2 sono:")
print(final_paramRR)

I migliori parametri in assoluto per modello1 sono: 
{'linearsvr__C': 0.1}
I migliori parametri in assoluto per modello2 sono:
{'ridge__alpha': 100}


In [ ]:
print("Errore di test per il modello migliore del modello1: %.2f" % (myscorer(final_modelSVR, X_new, y_new)))
print("Errore di test per il modello migliore del modello2: %.2f" % (myscorer(final_modelRR, X_new, y_new)))

Errore di test per il modello migliore del modello1: 11674.39
Errore di test per il modello migliore del modello2: 461.82
